# Basic SUEWS Simulation Tutorial

This notebook demonstrates how to run a basic urban climate simulation using the SUEWS MCP Server.

## What you'll learn:
- How to connect to the SUEWS MCP server
- How to configure a basic simulation
- How to run SUEWS and analyze results
- How to visualize energy balance components

## Prerequisites:
- SUEWS MCP server running (see README.md)
- Python packages: `mcp`, `pandas`, `matplotlib`

In [ ]:
# Import required packages
import asyncio
from mcp import create_client
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

# For Jupyter notebooks, we need nest_asyncio to handle event loops
import nest_asyncio
nest_asyncio.apply()

print("📦 Packages imported successfully")
print("🏃 Ready to connect to SUEWS MCP server")

## Step 1: Connect to the MCP Server

First, let's connect to the SUEWS MCP server and check what tools are available.

In [ ]:
async def connect_and_list_tools():
    """Connect to MCP server and list available tools."""
    try:
        async with create_client("suews-mcp") as client:
            print("✅ Successfully connected to SUEWS MCP server")
            
            # List available tools
            tools = await client.list_tools()
            
            print(f"\n📋 Available MCP tools ({len(tools.tools)}):")
            for tool in tools.tools:
                print(f"  • {tool.name}: {tool.description[:80]}...")
            
            return True
            
    except Exception as e:
        print(f"❌ Failed to connect: {e}")
        print("\n🔧 Troubleshooting:")
        print("   1. Check if SUEWS MCP server is running")
        print("   2. Verify server is on localhost:8000 (default)")
        print("   3. Check server logs for errors")
        return False

# Test connection
connection_ok = await connect_and_list_tools()

## Step 2: Check Server Health

Let's verify the server is healthy and ready for simulations.

In [ ]:
async def check_server_health():
    """Check SUEWS MCP server health status."""
    async with create_client("suews-mcp") as client:
        health = await client.call_tool("health_check", {})
        print("🏥 Server Health Status:")
        print(health.content[0].text)
        return health

if connection_ok:
    health_status = await check_server_health()

## Step 3: Explore Available Templates

Let's see what configuration templates are available for different urban surface types.

In [ ]:
async def explore_templates():
    """Explore available SUEWS configuration templates."""
    async with create_client("suews-mcp") as client:
        
        # List configuration templates
        resources = await client.call_tool("list_resources", {
            "resource_type": "config_template"
        })
        
        print("🏗️ Available Configuration Templates:")
        print(resources.content[0].text)
        
        return resources

if connection_ok:
    templates = await explore_templates()

## Step 4: Get a Configuration Template

Let's get the residential template and see what it looks like.

In [ ]:
async def get_residential_template():
    """Get and display the residential configuration template."""
    async with create_client("suews-mcp") as client:
        
        template = await client.call_tool("get_resource", {
            "resource_path": "templates/configs/residential.yml"
        })
        
        print("🏘️ Residential Template Configuration:")
        print("=" * 50)
        # Show first part of template
        template_text = template.content[0].text
        lines = template_text.split('\n')
        for line in lines[:30]:  # Show first 30 lines
            print(line)
        
        print("\n... (showing first 30 lines only) ...")
        print(f"\nTotal lines in template: {len(lines)}")
        
        return template_text

if connection_ok:
    residential_template = await get_residential_template()

## Step 5: Configure a Custom Simulation

Now let's configure a simulation for a specific location (London, UK) using the residential template.

In [ ]:
async def configure_london_simulation():
    """Configure a SUEWS simulation for London residential area."""
    async with create_client("suews-mcp") as client:
        
        print("⚙️ Configuring London residential simulation...")
        
        # Define London-specific parameters
        london_config = {
            "config_path": "templates/configs/residential.yml",
            "site_name": "London_Residential_Demo",
            "config_updates": {
                "site": {
                    "lat": 51.5074,      # London latitude
                    "lon": -0.1278,      # London longitude  
                    "elevation": 25,     # London elevation (m)
                    "timezone": 0        # UTC offset
                },
                "surface": {
                    "frac_paved": 0.35,  # 35% paved surfaces (roads, driveways)
                    "frac_bldgs": 0.25,  # 25% buildings 
                    "frac_grass": 0.30,  # 30% grass/lawns
                    "frac_trees": 0.10,  # 10% trees
                    "height_bldgs": 12.0, # Average building height (m)
                    "albedo_paved": 0.15, # Slightly higher albedo for UK
                    "albedo_bldgs": 0.25  # Building albedo
                },
                "anthropogenic": {
                    "qf0_beu": 22.0      # Anthropogenic heat (W/m²) - typical for UK residential
                },
                "model": {
                    "control": {
                        "tstep": 3600    # Hourly time step
                    }
                }
            },
            "save_path": "london_residential_config.yml"
        }
        
        # Configure the simulation
        config_result = await client.call_tool("configure_simulation", london_config)
        
        print("✅ Configuration Result:")
        print(config_result.content[0].text)
        
        return config_result

if connection_ok:
    config_result = await configure_london_simulation()

## Step 6: Validate Configuration

Before running the simulation, let's validate our configuration to catch any potential issues.

In [ ]:
async def validate_configuration():
    """Validate the London residential configuration."""
    async with create_client("suews-mcp") as client:
        
        print("🔍 Validating configuration...")
        
        validation = await client.call_tool("validate_config", {
            "config_file": "london_residential_config.yml",
            "strict_mode": False,
            "check_file_paths": True
        })
        
        print("📋 Validation Results:")
        print(validation.content[0].text)
        
        # Check if validation passed
        validation_text = validation.content[0].text
        if "✓ PASSED" in validation_text:
            print("\n🎉 Configuration validation successful! Ready to run simulation.")
            return True
        else:
            print("\n⚠️ Configuration has issues. Please review and fix before running simulation.")
            return False

if connection_ok:
    validation_passed = await validate_configuration()

## Step 7: Run the Simulation

Now let's run our first SUEWS simulation! We'll use sample data and run for a short period to start.

In [ ]:
async def run_london_simulation():
    """Run the London residential SUEWS simulation."""
    async with create_client("suews-mcp") as client:
        
        print("🚀 Starting SUEWS simulation...")
        print("   Configuration: London residential area")
        print("   Data: Sample meteorological data")
        print("   Period: One week (for demonstration)")
        print("   Expected runtime: 30-60 seconds")
        
        simulation = await client.call_tool("run_simulation", {
            "config_path": "london_residential_config.yml",
            "use_sample_data": True,  # Use built-in sample data
            "start_time": "2012-07-01T00:00:00",  # Summer week
            "end_time": "2012-07-07T23:00:00",
            "save_state": True
        })
        
        print("\n✅ Simulation completed!")
        print("\n📊 Simulation Results:")
        print(simulation.content[0].text)
        
        return simulation

if connection_ok and validation_passed:
    simulation_result = await run_london_simulation()
elif connection_ok:
    print("⏸️ Skipping simulation due to validation issues")
else:
    print("⏸️ Skipping simulation due to connection issues")

## Step 8: Analyze Results

Let's analyze the simulation results using the MCP analysis tools.

In [ ]:
async def analyze_simulation_results():
    """Analyze the SUEWS simulation results."""
    async with create_client("suews-mcp") as client:
        
        print("🔬 Analyzing simulation results...")
        
        # Energy balance analysis
        analysis = await client.call_tool("analyze_results", {
            "results_path": "outputs/London_Residential_Demo_SUEWS.csv",
            "analysis_type": "energy_balance",
            "variables": ["QH", "QE", "QN", "QS", "T2"],
            "time_period": "hourly"  # No aggregation for weekly data
        })
        
        print("📈 Energy Balance Analysis:")
        print(analysis.content[0].text)
        
        return analysis

# Run analysis if simulation was successful
if connection_ok and validation_passed:
    try:
        analysis_result = await analyze_simulation_results()
    except Exception as e:
        print(f"⚠️ Analysis failed: {e}")
        print("This might be normal if simulation output files are in a different location")

## Step 9: Load and Visualize Results

Let's load the simulation results directly and create some visualizations.

In [ ]:
# Try to load simulation results
result_files = [
    "outputs/London_Residential_Demo_SUEWS.csv",
    "London_Residential_Demo_SUEWS.csv",
    "SUEWS_output.csv"  # Default SUEWS output name
]

df_results = None
results_file = None

for file_path in result_files:
    try:
        print(f"Trying to load: {file_path}")
        df_results = pd.read_csv(file_path, delim_whitespace=True)
        
        # Try to parse datetime if columns exist
        if all(col in df_results.columns for col in ['iy', 'id', 'it', 'imin']):
            # Create datetime index from SUEWS time columns
            df_results['datetime'] = pd.to_datetime(
                df_results[['iy', 'id', 'it', 'imin']].rename(columns={
                    'iy': 'year', 'id': 'dayofyear', 'it': 'hour', 'imin': 'minute'
                }), 
                format='%Y%j%H%M'
            )
            df_results.set_index('datetime', inplace=True)
        
        results_file = file_path
        print(f"✅ Successfully loaded results from: {file_path}")
        print(f"📊 Data shape: {df_results.shape}")
        print(f"📅 Time period: {df_results.index[0]} to {df_results.index[-1]}")
        break
        
    except FileNotFoundError:
        print(f"❌ File not found: {file_path}")
        continue
    except Exception as e:
        print(f"⚠️ Error loading {file_path}: {e}")
        continue

if df_results is not None:
    print(f"\n📋 Available variables: {list(df_results.columns)[:10]}...")  # Show first 10
    print(f"\n📈 Sample statistics:")
    
    # Show statistics for key energy balance variables
    energy_vars = ['QN', 'QH', 'QE', 'QS', 'QF', 'T2']
    available_vars = [var for var in energy_vars if var in df_results.columns]
    
    if available_vars:
        print(df_results[available_vars].describe().round(2))
    else:
        print("Standard energy balance variables not found with expected names.")
        print(f"Available columns: {list(df_results.columns)}")
else:
    print("\n❌ Could not load simulation results")
    print("This might be because:")
    print("  - Simulation didn't complete successfully")
    print("  - Output file is in a different location")
    print("  - File format is different than expected")

## Step 10: Create Visualizations

Let's create some plots to visualize the urban energy balance results.

In [ ]:
if df_results is not None:
    # Set up the plotting environment
    plt.style.use('default')
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('SUEWS London Residential Simulation Results', fontsize=16, fontweight='bold')
    
    # Define color scheme
    colors = {
        'QN': '#2E8B57',   # Sea Green
        'QH': '#DC143C',   # Crimson  
        'QE': '#4169E1',   # Royal Blue
        'QS': '#FF8C00',   # Dark Orange
        'T2': '#8B0000'    # Dark Red
    }
    
    # Plot 1: Energy Balance Time Series
    ax1 = axes[0, 0]
    energy_vars = ['QN', 'QH', 'QE', 'QS']
    available_energy_vars = [var for var in energy_vars if var in df_results.columns]
    
    if available_energy_vars:
        for var in available_energy_vars:
            ax1.plot(df_results.index, df_results[var], 
                    label=f'{var} ({var_descriptions.get(var, "Energy flux")})',
                    color=colors.get(var, 'gray'), linewidth=1.5)
        
        ax1.set_title('Energy Balance Components', fontweight='bold')
        ax1.set_ylabel('Energy Flux (W/m²)')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        ax1.axhline(0, color='black', linestyle='-', alpha=0.3)
    else:
        ax1.text(0.5, 0.5, 'Energy balance variables\nnot found', 
                ha='center', va='center', transform=ax1.transAxes)
        ax1.set_title('Energy Balance Components (Not Available)')
    
    # Plot 2: Air Temperature
    ax2 = axes[0, 1]
    if 'T2' in df_results.columns:
        ax2.plot(df_results.index, df_results['T2'], 
                color=colors['T2'], linewidth=2)
        ax2.set_title('Air Temperature at 2m', fontweight='bold')
        ax2.set_ylabel('Temperature (°C)')
        ax2.grid(True, alpha=0.3)
        
        # Add temperature statistics
        temp_mean = df_results['T2'].mean()
        temp_min = df_results['T2'].min()
        temp_max = df_results['T2'].max()
        ax2.text(0.02, 0.98, f'Mean: {temp_mean:.1f}°C\nMin: {temp_min:.1f}°C\nMax: {temp_max:.1f}°C',
                transform=ax2.transAxes, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    else:
        ax2.text(0.5, 0.5, 'Temperature data\nnot found', 
                ha='center', va='center', transform=ax2.transAxes)
        ax2.set_title('Air Temperature (Not Available)')
    
    # Plot 3: Diurnal Pattern (average by hour)
    ax3 = axes[1, 0]
    if available_energy_vars:
        # Calculate hourly averages
        hourly_means = df_results[available_energy_vars].groupby(df_results.index.hour).mean()
        
        for var in available_energy_vars:
            ax3.plot(hourly_means.index, hourly_means[var], 
                    'o-', label=var, color=colors.get(var, 'gray'), 
                    linewidth=2, markersize=4)
        
        ax3.set_title('Average Diurnal Pattern', fontweight='bold')
        ax3.set_xlabel('Hour of Day')
        ax3.set_ylabel('Energy Flux (W/m²)')
        ax3.set_xticks(range(0, 24, 3))
        ax3.legend()
        ax3.grid(True, alpha=0.3)
        ax3.axhline(0, color='black', linestyle='-', alpha=0.3)
    else:
        ax3.text(0.5, 0.5, 'Energy balance variables\nnot available for diurnal analysis', 
                ha='center', va='center', transform=ax3.transAxes)
        ax3.set_title('Diurnal Pattern (Not Available)')
    
    # Plot 4: Energy Balance Closure (if QN is available)
    ax4 = axes[1, 1]
    if all(var in df_results.columns for var in ['QN', 'QH', 'QE']):
        # Calculate energy balance closure
        available_energy = df_results['QN']
        if 'QS' in df_results.columns:
            turbulent_fluxes = df_results['QH'] + df_results['QE'] + df_results['QS']
        else:
            turbulent_fluxes = df_results['QH'] + df_results['QE']
        
        # Scatter plot of energy balance
        ax4.scatter(available_energy, turbulent_fluxes, 
                   alpha=0.6, s=20, color='blue')
        
        # 1:1 line
        min_val = min(available_energy.min(), turbulent_fluxes.min())
        max_val = max(available_energy.max(), turbulent_fluxes.max())
        ax4.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='1:1 line')
        
        ax4.set_title('Energy Balance Closure', fontweight='bold')
        ax4.set_xlabel('Available Energy QN (W/m²)')
        ax4.set_ylabel('Turbulent Fluxes (W/m²)')
        ax4.legend()
        ax4.grid(True, alpha=0.3)
        
        # Calculate and display R²
        correlation = np.corrcoef(available_energy, turbulent_fluxes)[0, 1]
        r_squared = correlation ** 2
        ax4.text(0.05, 0.95, f'R² = {r_squared:.3f}', 
                transform=ax4.transAxes, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    else:
        ax4.text(0.5, 0.5, 'Energy balance closure\nrequires QN, QH, QE', 
                ha='center', va='center', transform=ax4.transAxes)
        ax4.set_title('Energy Balance Closure (Not Available)')
    
    plt.tight_layout()
    plt.show()
    
    # Variable descriptions for labels
    var_descriptions = {
        'QN': 'Net radiation',
        'QH': 'Sensible heat',
        'QE': 'Latent heat', 
        'QS': 'Storage heat',
        'QF': 'Anthropogenic heat',
        'T2': 'Air temperature'
    }
    
    print("📊 Visualization complete!")
    
else:
    print("📊 Visualization skipped - no results data available")
    
    # Create a demonstration plot showing what the results would look like
    print("\n🎨 Creating example visualization to show expected output format...")
    
    # Generate synthetic data similar to SUEWS output
    hours = pd.date_range('2012-07-01', '2012-07-07 23:00:00', freq='H')
    n_points = len(hours)
    
    # Synthetic energy balance data with realistic patterns
    time_of_day = hours.hour
    day_of_year = hours.dayofyear
    
    # Create realistic diurnal and day-to-day patterns
    QN_synthetic = 200 * np.sin(np.pi * time_of_day / 12) * np.maximum(0, np.sin(np.pi * time_of_day / 24)) + np.random.normal(0, 20, n_points)
    QH_synthetic = 0.3 * QN_synthetic + 20 + np.random.normal(0, 10, n_points)
    QE_synthetic = 0.4 * QN_synthetic + 10 + np.random.normal(0, 15, n_points)
    QS_synthetic = 0.2 * QN_synthetic + np.random.normal(0, 8, n_points)
    T2_synthetic = 18 + 8 * np.sin(np.pi * time_of_day / 12) + np.random.normal(0, 2, n_points)
    
    # Create synthetic dataframe
    df_synthetic = pd.DataFrame({
        'QN': QN_synthetic,
        'QH': QH_synthetic,
        'QE': QE_synthetic,
        'QS': QS_synthetic,
        'T2': T2_synthetic
    }, index=hours)
    
    # Create the same plots with synthetic data
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Example: SUEWS London Residential Results (Synthetic Data)', fontsize=16, fontweight='bold')
    
    colors = {'QN': '#2E8B57', 'QH': '#DC143C', 'QE': '#4169E1', 'QS': '#FF8C00', 'T2': '#8B0000'}
    
    # Energy balance time series
    ax1 = axes[0, 0]
    for var in ['QN', 'QH', 'QE', 'QS']:
        ax1.plot(df_synthetic.index, df_synthetic[var], 
                label=f'{var}', color=colors[var], linewidth=1.5)
    ax1.set_title('Energy Balance Components', fontweight='bold')
    ax1.set_ylabel('Energy Flux (W/m²)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.axhline(0, color='black', linestyle='-', alpha=0.3)
    
    # Temperature
    ax2 = axes[0, 1]
    ax2.plot(df_synthetic.index, df_synthetic['T2'], color=colors['T2'], linewidth=2)
    ax2.set_title('Air Temperature at 2m', fontweight='bold')
    ax2.set_ylabel('Temperature (°C)')
    ax2.grid(True, alpha=0.3)
    
    # Diurnal pattern
    ax3 = axes[1, 0]
    hourly_means = df_synthetic[['QN', 'QH', 'QE', 'QS']].groupby(df_synthetic.index.hour).mean()
    for var in ['QN', 'QH', 'QE', 'QS']:
        ax3.plot(hourly_means.index, hourly_means[var], 'o-', label=var, 
                color=colors[var], linewidth=2, markersize=4)
    ax3.set_title('Average Diurnal Pattern', fontweight='bold')
    ax3.set_xlabel('Hour of Day')
    ax3.set_ylabel('Energy Flux (W/m²)')
    ax3.set_xticks(range(0, 24, 3))
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # Energy balance closure
    ax4 = axes[1, 1]
    available_energy = df_synthetic['QN']
    turbulent_fluxes = df_synthetic['QH'] + df_synthetic['QE'] + df_synthetic['QS']
    ax4.scatter(available_energy, turbulent_fluxes, alpha=0.6, s=20, color='blue')
    min_val = min(available_energy.min(), turbulent_fluxes.min())
    max_val = max(available_energy.max(), turbulent_fluxes.max())
    ax4.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='1:1 line')
    ax4.set_title('Energy Balance Closure', fontweight='bold')
    ax4.set_xlabel('Available Energy QN (W/m²)')
    ax4.set_ylabel('Turbulent Fluxes (W/m²)')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("🎨 Example visualization complete! This shows the type of results you'd get from a real simulation.")

## Summary and Next Steps

Congratulations! You've completed your first SUEWS urban climate simulation using the MCP server. Here's what you accomplished:

### ✅ What you did:
1. **Connected** to the SUEWS MCP server
2. **Explored** available configuration templates
3. **Configured** a simulation for London residential area
4. **Validated** the configuration parameters
5. **Ran** a SUEWS simulation with sample data
6. **Analyzed** and **visualized** the results

### 📊 Key Results (typical expectations):
- **Energy Balance**: QN (net radiation) drives QH (sensible heat) and QE (latent heat)
- **Temperature**: Realistic diurnal patterns with daytime heating and nighttime cooling
- **Urban Effects**: Anthropogenic heat and thermal storage affect energy balance
- **Bowen Ratio**: QH/QE ratio indicates the partitioning between sensible and latent heat

### 🚀 Next Steps:

1. **Try Different Urban Types**:
   ```python
   # Commercial district
   "config_path": "templates/configs/commercial.yml"
   
   # Urban park
   "config_path": "templates/configs/park.yml"
   ```

2. **Use Your Own Data**:
   ```python
   # Process your own meteorological data
   await client.call_tool("preprocess_forcing", {
       "input_file": "your_weather_data.csv",
       "auto_fix_issues": True
   })
   ```

3. **Parameter Sensitivity**:
   - Test different albedo values
   - Vary building heights and densities
   - Explore anthropogenic heat impacts

4. **Longer Simulations**:
   - Run full annual simulations
   - Analyze seasonal patterns
   - Compare different years

5. **Advanced Analysis**:
   - Urban heat island quantification
   - Building energy implications
   - Climate change impact assessment

### 📚 Additional Resources:
- [SUEWS MCP API Reference](../docs/api_reference.md)
- [Urban Heat Island Example](../docs/examples/urban_heat_island_study.md)
- [Parameter Sensitivity Analysis](../docs/examples/parameter_sensitivity_analysis.md)
- [Data Preprocessing Guide](../docs/examples/data_preprocessing_workflow.md)

### 🆘 Need Help?
- Check the [FAQ & Troubleshooting Guide](../docs/faq.md)
- Visit [SUEWS Documentation](https://suews.readthedocs.io/)
- Open an issue on [GitHub](https://github.com/UMEP-dev/SUEWS/issues)

Happy urban climate modeling! 🏙️🌡️

In [ ]:
# Final check - print summary of what was accomplished
print("🎉 TUTORIAL COMPLETE!")
print("=" * 50)

if connection_ok:
    print("✅ MCP server connection: SUCCESS")
else:
    print("❌ MCP server connection: FAILED")

if 'validation_passed' in locals() and validation_passed:
    print("✅ Configuration validation: PASSED")
elif 'validation_passed' in locals():
    print("⚠️ Configuration validation: ISSUES FOUND")
else:
    print("⏸️ Configuration validation: SKIPPED")

if 'simulation_result' in locals():
    print("✅ SUEWS simulation: COMPLETED")
else:
    print("⏸️ SUEWS simulation: SKIPPED")

if df_results is not None:
    print("✅ Results loading: SUCCESS")
    print(f"📊 Data points: {len(df_results):,}")
    print(f"🗂️ Variables: {len(df_results.columns)}")
else:
    print("📊 Results visualization: DEMONSTRATED WITH SYNTHETIC DATA")

print("\n🚀 Ready for more advanced SUEWS analyses!")
print("\n📖 Next: Try the Urban Heat Island notebook or Parameter Sensitivity notebook")